### This notebook demonstrates two methods for connecting to Postgres.

The first is [iPython-sql](https://pypi.python.org/pypi/ipython-sql) a "magic" for Jupyter Notebook (what this document is written in).

The second is [Psycopg](http://initd.org/psycopg/), which is the fundamental Postgres connector for Python.

In [ ]:
%load_ext sql

In [ ]:
%sql postgresql:///decibel

In [ ]:
%sql SELECT now()

In [ ]:
import psycopg2
db = psycopg2.connect('')
db2 = psycopg2.connect('')

In [ ]:
db

In [ ]:
db2

In [ ]:
db is db2

In [ ]:
%sql SET application_name = 'ipython-sql'
db.cursor().execute("SET application_name = 'psycopg-1'")
db2.cursor().execute("SET application_name = 'psycopg-2'")
db.commit()
db2.commit()

Because the `%sql` magic uses SQLalchemy under the hood, it can end up in a transaction, which means you might get stale data. If that happens, you want to issue either a `ROLLBACK;` or `COMMIT;`.

In [ ]:
%sql BEGIN;
%sql SELECT count(*) FROM pg_stat_activity
db.cursor().execute('CREATE TEMP TABLE i(i int)')

In [ ]:
%sql SELECT pid,datname,usename,application_name,state_change,state,query FROM pg_stat_activity

In [ ]:
%%sql
ROLLBACK;
SELECT pid,datname,usename,application_name,state_change,state,query FROM pg_stat_activity;

In [ ]:
%sql SELECT * FROM pg_tables WHERE schemaname LIKE 'pg_temp%'

In [ ]:
db.commit()

In [ ]:
%sql SELECT * FROM pg_tables WHERE schemaname LIKE 'pg_temp%'

## How to execute queries using psycopg2?

First, lets grap a psycopg2 "cursor". This is *not* the same as a database cursor!

In [ ]:
c = db.cursor()

Run a command with the cursor:

In [ ]:
c.execute("SELECT * FROM pg_tables LIMIT 5") # Note the LIMIT 5

How to fetch results?

In [ ]:
r = c.fetchone()
r

In [ ]:
for result in c: # Note that c is our cursor
    print result

In [ ]:
SQLactivity="SELECT pid,datname,usename,application_name,state_change,state,query FROM pg_stat_activity"
c.execute(SQLactivity)

In [ ]:
rows = c.fetchmany(2)
rows

In [ ]:
len(rows)

In [ ]:
c.statusmessage # Command status, same as you'd see in psql

In [ ]:
c.rownumber

In [ ]:
c.rowcount

In [ ]:
len(c.fetchall())

`%sql` can return a result object

In [ ]:
r = %sql SELECT schemaname,count(*) FROM pg_tables GROUP BY schemaname

In [ ]:
r

How about a DataFrame?

In [ ]:
import pandas as pd

df = r.DataFrame()

df

and plots/graphs

In [ ]:
%matplotlib inline

r.pie()

`%sql` also makes it easy to create new tables (but `CREATE TABLE ... AS SELECT ...` would be far more efficmient)

In [ ]:
%sql PERSIST df

In [ ]:
%sql SELECT * FROM df

Note the difference from the original results...

In [ ]:
r

You can only persist a DataFrame though...

In [ ]:
%sql PERSIST r

What do psycopg results look like?

In [ ]:
c.execute('SELECT * FROM df')
c_df_result = c.fetchall()
c_df_result

In [ ]:
type(c_df_result)

In [ ]:
type(c_df_result[0])

How do you know what's in the result?

In [ ]:
c.description

In [ ]:
import psycopg2.extras

# Note this is a new cursor
c2 = db.cursor(cursor_factory=psycopg2.extras.DictCursor)
c2.execute('SELECT * FROM df')
c2_df_result = c2.fetchall()
c2_df_result

In [ ]:
c_df_result

In [ ]:
type(c2_df_result[0])

In [ ]:
row = c2_df_result[0]

In [ ]:
row

In [ ]:
c_df_result[0]

Individual `DictRow` results behave like a combination list and dict.

In [ ]:
row.keys()

In [ ]:
row[1]

In [ ]:
row['schemaname']

In [ ]:
row['schemaname'] is row[1]

Remember that results are static once they've been executed.

In [ ]:
# Add 10 to each index
c2.execute('UPDATE df SET index=index*10')

In [ ]:
c2_df_result

In [ ]:
c2.execute('SELECT * FROM df')

In [ ]:
c2.fetchall()

What if that's not what I wanted to do?

In [ ]:
db.autocommit

In [ ]:
db.rollback()
c.execute('UPDATE df SET index=index+10')
c2.execute('SELECT * FROM df') # different cursor! 
c2.fetchall()

`%sql` can't see these updates yet though...

In [ ]:
%sql SELECT * FROM df

In [ ]:
db2c = db2.cursor()
db2c.execute('SELECT * FROM df')
db2c.fetchall()

In [ ]:
c2.commit()

What if all you have is the cursor?

In [ ]:
c2.connection.commit()
c2.connection is db

In [ ]:
%sql SELECT * FROM df

`c2_df_result` is unchanged though...

In [ ]:
c2_df_result

We can easily turn `psycopg` tuple results into a dataframe, but the dataframe won't have any names.

In [ ]:
c.execute(SQLactivity)
pd.DataFrame(c.fetchall())

Why no names?

In [ ]:
c.execute(SQLactivity)
type(c.fetchone())

Results from `c2` would have names associated with them!

In [ ]:
c2.execute(SQLactivity)
pd.DataFrame([row for row in c2])

In [ ]:
c2.execute(SQLactivity)
pd.DataFrame([dict(row) for row in c2])

Pandas can also execute SQL directly.

In [ ]:
new_df=pd.read_sql(SQLactivity, db)
new_df

Notice the ordering of the dataframe columns is different, because the ordering of keys in an ordinary `dict` is indeterminent.

In [ ]:
# Lets create a new table...
c2.execute('CREATE TABLE df2(s serial, index int, schemaname name, count bigint)')

In [ ]:
# And insert a row into it
SQLdf2_insert='''INSERT INTO df2 VALUES(
    DEFAULT -- Use default value for s
    , %(index)s, %(schemaname)s, %(count)s -- We will provide these values *via psycopg2*
    )'''

c2.execute(SQLdf2_insert, c2_df_result[0]) # c2_df_result[0] is the first row of the result set

In [ ]:
%sql SELECT * FROM df2

In [ ]:
db.commit()

In [ ]:
%sql SELECT * FROM df2

`psycopg` supports multiple-execution.

In [ ]:
c.executemany(SQLdf2_insert, c2_df_result) # Note executemany(), not just execute(), and no array index
pd.read_sql('SELECT * FROM df2', db)

In [ ]:
c.execute('SELECT * FROM df2')
c.fetchall()

In [ ]:
c3=db.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor)
c3.execute('SELECT * FROM df2')
r=c3.fetchall()
r

In [ ]:
r[0]

In [ ]:
r[0][2]

In [ ]:
r[0].schemaname

In [ ]:
r[0][2] is r[0].schemaname

In [ ]:
with db:
    c3.execute(SQLdf2_insert + ' RETURNING *', r[0]._asdict())
    r=c3.fetchall()

In [ ]:
r

In [ ]:
%sql SELECT * FROM df2

Why does that return everything? Because we used `db` as a context manager.

In [ ]:
c.execute('BAD SQL')

In [ ]:
c.execute('SELECT 1')

In [ ]:
db.rollback() # A rollback on the connection puts us back in a clean state
c.execute('SELECT 1')
c.fetchall()

What if we run bad SQL in a context manager?

In [ ]:
with db:
    c.executemany(SQLdf2_insert, c2_df_result)
    c.execute('BAD SQL')
    c.execute('SELECT 1')
    c.fetchall()

Will this fail?

In [ ]:
c.execute('SELECT 1')
c.fetchall()

You can use cursors as context managers too.

In [ ]:
with db.cursor() as new_c:
    new_c.execute('SELECT * FROM df2')
    r=new_c.fetchall()

In [ ]:
r

In [ ]:
new_c.execute('SELECT 1')

In [ ]:
new_c

In [ ]:
c

In [ ]:
new_c.closed

In [ ]:
%%sql
CREATE EXTENSION IF NOT EXISTS plpythonu;
CREATE FUNCTION add(
    v1 int
    , v2 int
) RETURNS text LANGUAGE plpythonu AS $$
return v1 + v2
$$;
SELECT add(1, 2);

In [ ]:
%%sql
DO LANGUAGE plpythonu $$
plpy.execute('DROP FUNCTION add(int,int)')
$$;

In [ ]:
%sql SELECT add(1,1)

## Questions?
#### Jim.Nasby@BlueTreble.com

#### This notebook can be downloaded from https://github.com/AustinPUG/PGDay2016/blob/master/Python%2BPostgres.ipynb (search for PGDay2016 on Github)